In [1]:
%load_ext Cython

In [2]:
import numpy as np 
import  matplotlib.pyplot as plt 
%matplotlib notebook 
# last line enables interactive plotting in notebooks 

In [3]:
#!python

# -------- these are not just comments. These are compiler directives. 
# They make code run a bit faster. You should take them away for debug/development
# (if you use wraparound=False you can't use negative number for indexing arrays from the end)
#
#cython: boundscheck=False
#cython: wraparound=False
#cython: initializedcheck=True

import numpy as np
import cython  




cohesinAmount = 10
rnapAmount = 5
baseNumber = 500
geneNumber = 10
maxLength = 15
sisterOne = np.empty(cohesinAmount + rnapAmount, dtype=int)
sisterTwo = np.empty(cohesinAmount + rnapAmount, dtype=int)


def isOccupied(array, place):
    """
    Checks if a place on a chromatid is occupied by a protien
    
    Parameters
    ----------
    array : one of the two chromatids
    place : location on a chromatid
    
    Returns:
    --------
    true if the place is occupied and false if it is not occupied
    """
    return (array == place).any()

def extendedOccupied(array, place, length):
    """
    Checks if a segment on a chromatid is occupied by any protien
    
    Parameters
    ----------
    array : one of the two chromatids
    place : location on a chromatid
        Gives the starting point of the segment
    length: integer
        Gives the length of the segment
    
    Returns:
    --------
    true if the segment has any protiens and false if it is empty
    """
    return ((array >= place) * (array < (place + length))).any()

def checkProtien(array, place):
    """
    Finds the location of a protien on a chromatid
    
    Parameters
    ----------
    array : one of the two chromatids
    place : location on a chromatid
    
    Returns:
    --------
    index on the chromatid array of the protien which is in the set location
    """
    return np.where(array == place)[0]
    
def generateGeneLengths():
    """
    Generates the legnths of all the genes

    Returns:
    --------
    an array of lengths for all the genes
    """
    geneLengths = np.empty(geneNumber, dtype=int)
    for gene in range(0, geneNumber):
        geneLengths[gene] = np.random.randint(5,16)
    return geneLengths

def generateShadowGenes(geneLengths):
    """
    Generates the lengths of the empty spaces between genes
    
    Paramaters:
    ----------
    geneLengths : an array given by generateGeneLengths()
        This is needed to know how long the sum of the shadow lengths is
    
    Returns:
    --------
    array of 'shadow genes'
    """
    finalSum = 500 - sum(geneLengths)
    shadowGenes = np.empty(geneNumber + 1, dtype=int)
    pseudoShadow = np.empty(geneNumber + 1, dtype=float)
    for shadowIndex in range(0, geneNumber + 1):
        pseudoShadow[shadowIndex] = np.random.random()
    pseudoLength = sum(pseudoShadow)
    for shadow in range(0, geneNumber + 1):
        pseudoShadow[shadow] = pseudoShadow[shadow]/pseudoLength
        shadowGenes[shadow] = round(finalSum*pseudoShadow[shadow], 0)
    return shadowGenes

def generateDirection():
    """
    Generates an array of gene directions

    Returns:
    --------
    an array in which -1 means the gene is going backwards and 1 means it is going formwards
    """
    geneDirection = np.empty(geneNumber, dtype=int)
    for gene in range(0, geneNumber):
        geneFlip = np.random.random()
        if geneFlip < 0.5:
            geneDirection[gene] = -1
        else:
            geneDirection[gene] = 1
    return geneDirection
    
def generateGenes(geneLengths, shadowGenes):
    """
    Generates genes without direction
    
    Parameters
    ----------
    geneLengths : The array given by generateGeneLenths()
    shadowGenes : The array given by generateShadowGenes()
    
    Returns:
    --------
    an array which holds the starting points of each gene assuming no direction
    """
    geneList = np.empty(geneNumber, dtype=int)
    geneList[0] = shadowGenes[0]
    geneStart = shadowGenes[0]
    for gene in range(0, geneNumber-1):
        geneStart = geneStart+geneLengths[gene]+shadowGenes[gene+1]
        geneList[gene+1] = geneStart
    return geneList

def flipGenes(geneList, geneDirection, geneLength):
    """
    Flips the genes which have a negative direction
    
    Parameters
    ----------
    geneList : The array generated by generateGenes()
    geneDirection : The array generated by generateGeneDirection()
    geneLength : The array given by generateGeneLenths()
      
    Returns:
    --------
    an array which holds the starting points of each gene
    """
    for gene in range(0, geneNumber):
        if geneDirection[gene] == -1:
            geneList[gene] = geneList[gene] + geneLength[gene]
    return geneList

def generateFullDNA(geneList, geneDirection, geneLength):
    """
    Generates the full DNA information
    
    Parameters
    ----------
    geneList : The array given by generateGenes()
    geneDirection : The array generated by generateGeneDirection()
    geneLengths : The array given by generateGeneLenths()
    
    
    Returns:
    --------
    an array in which for every base which is part of a gene it has 1 if the gene is going forward and -1 if it is going backwards
    """
    fullDNA = np.empty(baseNumber, dtype = int)
    for base in range(0, baseNumber):
        fullDNA[base] = 0
    for gene in range(0, geneNumber):
        if geneDirection[gene] == 1:
            for base in range(0, geneLength[gene]):
                fullDNA[geneList[gene] + base] = 1
        else:
            for base in range(0, geneLength[gene]):
                fullDNA[geneList[gene] - base] = -1
    return fullDNA

def generateCohesins(array):
    """
    Generates cohesin locations
    
    Parameters
    ----------
    array : a sister chromatid
    
    Returns:
    --------
    an array which contains the information about the lovations of cohesins on a chromatid
    """
    cohesin = 0
    while cohesin < cohesinAmount:
        place = np.random.randint(0, baseNumber)
        if not isOccupied(array, place):
            array[rnapAmount+cohesin] = place
            cohesin = cohesin + 1
    return array

def generateRnaps(array, geneList):
    """
    Generates RNAP locations
    
    Parameters
    ----------
    array : a sister chromatid
    geneList : the array generated by flipGenes()
    
    Returns:
    --------
    an array which contains the information about the lovations of RNAPs on a chromatid
    """
    rnap = 0
    while rnap < rnapAmount:
        gene = np.random.randint(0, 10)
        if not isOccupied(array, geneList[gene]):
            array[rnap] = geneList[gene]
            rnap = rnap + 1
    return array
    
    




In [4]:
geneLengths = generateGeneLengths()
shadowGenes = generateShadowGenes(geneLengths)
geneDirections = generateDirection()
geneList = generateGenes(geneLengths, shadowGenes)
geneList = flipGenes(geneList, geneDirections, geneLengths)
fullDNA = generateFullDNA(geneList, geneDirections, geneLengths)

print("shadow gene lengths: " , shadowGenes)
print("gene lengths: " , geneLengths)
print("gene directions: " , geneDirections)
print("gene list: " , geneList)
print(fullDNA)




shadow gene lengths:  [34 38 76 62  9 33 11  5 25 46 47]
gene lengths:  [ 9 11 10 14 12  6 11 13 13 14]
gene directions:  [-1 -1  1  1 -1 -1  1 -1  1 -1]
gene list:  [ 43  92 168 240 275 314 325 354 379 452]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  1  1  1  1  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  0  0  0  0  0  0  

In [9]:
speedRnap = .75
speedCohesin = .5

def stepRnap(array, fullDNA):
    """
    Moves all the RNAPs along there genes
    
    Parameters
    ----------
    array : a sister chromatid
    fullDNA : array generated by generateFullDNA()
    
    Returns:
    --------
    an array which contains the information about the locations of RNAPS on a chromatid after a step
    """
    movingRnap = 0
    speed = speedRnap
    while movingRnap < rnapAmount:
        if fullDNA[array[movingRnap]] == 0:
            gene = np.random.randint(0, 10)
            if not isOccupied(array, geneList[gene]):
                array[movingRnap] = geneList[gene]
                movingRnap = movingRnap + 1
        else:
            movementCheck = np.random.random()
            if movementCheck < speed:
                if not isOccupied(array, array[movingRnap] + fullDNA[array[movingRnap]]):
                    array[movingRnap] = array[movingRnap] + fullDNA[array[movingRnap]]
                    movingRnap = movingRnap + 1
                else:
                    array[checkProtien(array, array[movingRnap] + fullDNA[array[movingRnap]])] = array[movingRnap] + 2*fullDNA[array[movingRnap]]
                    array[movingRnap] = array[movingRnap] + fullDNA[array[movingRnap]]
                    movingRnap = movingRnap + 1
                    print("push")
            else:
                movingRnap = movingRnap + 1
    return array
            
def stepCohesin(array):
    """
    Moves all the cohesinss along there genes
    
    Parameters
    ----------
    array : a sister chromatid
    fullDNA : array generated by generateFullDNA()
    
    Returns:
    --------
    an array which contains the information about the locations of cohesins on a chromatid after a step
    """
    movingCohesin = 0
    speed = speedCohesin
    while movingCohesin < cohesinAmount:
        movementCheck = np.random.random()
        if movementCheck > speed:
            movingCohesin = movingCohesin + 1
        elif movementCheck > speed/2:
            if array[rnapAmount + movingCohesin] < baseNumber:
                if not isOccupied(array, array[rnapAmount + movingCohesin] + 1):
                    array[rnapAmount + movingCohesin] = array[rnapAmount + movingCohesin] + 1
            movingCohesin = movingCohesin + 1
        else:
            if array[rnapAmount + movingCohesin] > 0:
                if not isOccupied(array, array[rnapAmount + movingCohesin] - 1):
                    array[rnapAmount + movingCohesin] = array[rnapAmount + movingCohesin] - 1
            movingCohesin = movingCohesin + 1
    return array

def fullStep(array, fullDNA):
    """
    Moves all the protiens along there genes
    
    Parameters
    ----------
    array : a sister chromatid
    fullDNA : array generated by generateFullDNA()
    
    Returns:
    --------
    an array which contains the information about the locations of protiens on a chromatid after a step
    """
    array = stepRnap(array, fullDNA)
    array = stepCohesin(array)
    return array


In [10]:
print(geneList)
print(geneDirections)
sisterOne = generateCohesins(sisterOne)
sisterTwo = np.copy(sisterOne)
sisterOne = generateRnaps(sisterOne, geneList)
sisterTwo = generateRnaps(sisterTwo, geneList)
print("sisterOne: " , sisterOne)
print("sisterTwo: " , sisterTwo)
for itteration in range(0, 10):
    print("sisterOne: " , sisterOne)
    print("sisterTwo: " , sisterTwo)
    sisterOne = fullStep(sisterOne, fullDNA)
    sisterTwo = fullStep(sisterTwo, fullDNA)



[ 43  92 168 240 275 314 325 354 379 452]
[-1 -1  1  1 -1 -1  1 -1  1 -1]
sisterOne:  [379  43 325 314 354 470  27 468 475 343 253 452 322  33 390]
sisterTwo:  [ 43 314 379 354 325 470  27 468 475 343 253 452 322  33 390]
sisterOne:  [379  43 325 314 354 470  27 468 475 343 253 452 322  33 390]
sisterTwo:  [ 43 314 379 354 325 470  27 468 475 343 253 452 322  33 390]
sisterOne:  [380  42 325 313 353 470  26 469 475 342 252 452 322  32 389]
sisterTwo:  [ 42 313 380 353 326 470  27 468 476 343 253 452 322  33 391]
sisterOne:  [381  42 326 312 353 471  26 469 475 342 253 453 323  32 389]
sisterTwo:  [ 41 312 381 352 326 471  26 468 475 344 252 453 322  34 392]
sisterOne:  [381  41 327 312 353 470  27 469 476 342 254 452 322  33 390]
sisterTwo:  [ 40 311 382 351 326 471  25 468 475 344 252 453 323  34 392]
sisterOne:  [381  40 328 311 352 470  27 468 476 343 255 451 321  33 390]
sisterTwo:  [ 39 311 383 350 326 472  25 469 475 343 251 452 324  34 392]
sisterOne:  [382  39 329 310 352 469  